
Photo by Skitterphoto from Pexels

## The neighbourhoods of Groningen 
### Using Python and the folium mapping library to create an interactive map

This example shows how to convert a Shape file to a GEOJson format using the geopandas library and then plot an interactive map using the folium mapping library

**Geopandas** 
- A geopandas dataframe is similar to a pandas dataframe with some addtional geospatial features:
   - Using the geopandas read_file method to import a Shape file into a geopandas dataframe
   - Store specfic geospatial figures as polygon's or point's (longitude and latitude pairs)
   - Convert one geo-coordinate system to another 
   - Export to a GEOJson format
   - Methods to calculate a distance between two points or calculate an area of a polygon
- A geopandas dataframe has all the features of a pandas dataframe which makes it easy to manipulate
 
**Folium**
- The folium library has the Choropleth class which enables you to create an interactive map 
- The map can have the following functionality:
  - zoom-in/zoom-out
  - color coding of intensities, for example showing the population density of a neighbourhood<br>
    a lower population density having a lighter color or shade and higher population density showing<br>
    as a darker color or shade
  - adding a title and a legend
- More examples can be found [here](https://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/examples/)

Here we will be using a GEOJson file and a geopandas dataframe to plot the population densities by neighbourhood



In [1]:
import geopandas as gpd
import folium 
import numpy as np
import matplotlib.pyplot as plt

### Steps to convert a ESRI-Shape file to a geopandas dataframe and GEOJson format
1. Download the [zip](https://www.cbs.nl/nl-nl/dossier/nederland-regionaal/geografische%20data/wijk-en-buurtkaart-2017) file from the CBS (Centraal Bureau voor de Statistieken) containing the Wijk- en Buurtkaart 2017.<br>
   This file contains all the neighbourhoods of cities and towns in the Netherlands<br>
   Note: parts of the data from CBS has also been provided by the Kadaster
2. Unzip the file which will contain several files.<br>
   We are looking for the ones with a buurt_2017 prefix. The main file is buurt_2017.shp<br>
   This file contains the boundaries of each neighbourhood stored in a polygon format<br>
   a collection latitude/longitude points
3. Import the buurt_2017.shp file into a geopandas dataframe using the read_file function
4. We only want the neighbourhoods of the city of Gronigen so create a filter and rebuild the dataframe
5. The folium map coordinates system is based on the WSG84 system also know as egsg:4326<br>
   We need to convert the existing format "rijksdriehoekcoordinaten format" (epsg:28992) to WSG84<br>
   the to_crs method will do the job
6. For this example we only need a few columns so only select the following<br>
   - BU_CODE = neighbourhood code
   - BU_NAAM = neighbourhood name,
   - BEV_DICHTH = population density
   - AANT_INW = number of inhabitants
   - **Note** for more information (in Dutch) on the columns see the accompanying [pdf](https://www.cbs.nl/-/media/cbs/dossiers/nederland%20regionaal/wijk-en-buurtstatistieken/2018/2018ep42%20toelichting%20buurtkaart%202018.pdf)<br>
7. There are several neighbourhoods that have a negative population density, like parks<br>
   Correct these values and set to 0
8. Determine the center geo-coordinates of each neighbourhood using the geopandas centroid function<br>
   We need this to display a marker for each neighbourhood on the map
   

In [2]:
# 3. Reading the Netherlands neighbourhoods shape file into a geopandas dataframe
df_grun = gpd.read_file('data/buurt_2017.shp')

# 4. Only interested in Gronigen
df_grun = df_grun[df_grun["GM_NAAM"] == "Groningen"]

# 5. Convert geometry from rijksdriehoekcoordinaten format (epsg:28992) 
# to WSG84 (epsg:4326) format in order to display in a folium choropleth map
df_grun = df_grun.to_crs({'init' :'epsg:4326'})

# 6. We only need a few columns: 
# - BU_CODE = neighbourhood code
# - BU_NAAM = neighbourhood name,
# - BEV_DICHTH = population density
# - AANT_INW = number of inhabitants
df_grun = df_grun[['BU_CODE','BU_NAAM','BEV_DICHTH','AANT_INW','geometry']]

# 7. Fix negative population density as there are area's with -9999 (parks etc.)
df_grun["BEV_DICHTH"] = np.where((df_grun.BEV_DICHTH < 0), 0, df_grun.BEV_DICHTH)

# 8. Determine the centroid of each neighbourhood for display purposes
df_grun['Latitude']  = df_grun['geometry'].centroid.y
df_grun['Longitude'] = df_grun['geometry'].centroid.x

# have a look at the file
df_grun.head()

,BU_CODE,BU_NAAM,BEV_DICHTH,AANT_INW,geometry,Latitude,Longitude
73,BU00140000,Binnenstad-Noord,11974,4440,"POLYGON ((6.557103363006571 53.21932803109325,...",53.219465,6.564796
74,BU00140001,Binnenstad-Zuid,11977,6600,"POLYGON ((6.572740733905065 53.21845950105196,...",53.215108,6.567355
75,BU00140002,Binnenstad-Oost,14610,3920,"POLYGON ((6.566275867494596 53.22224113545024,...",53.219609,6.574328
76,BU00140003,Binnenstad-West,18101,1765,"POLYGON ((6.556893850524609 53.21930263843799,...",53.216044,6.557816
77,BU00140004,Noorderplantsoen,0,10,"POLYGON ((6.56022821781004 53.22768365527901, ...",53.223439,6.555731


Use the geopandas to_file method to create a GeoJson file which we will need to display the map

In [3]:
# convert to GeoJSON format
df_grun.to_file("data/groningen_buurten.json", driver="GeoJSON")

## Create and display the folium choropleth map 
1. Used google maps to determine the central coordinates of the city<br>
   and with some trial and error determine the best zoom factor to be 13<br>
   in order to display the map at the prefered size.<br>
   Use the folium `Map` class to create the initial map object
2. Use the folium `Choropleth` class to create a map based on the GeoJson file we have just create<br>
   The map will used the geo-coordinates in the file to display the boundaries of each neighbourhood<br>
   **Method attributes**
   - `geo_data="data/groningen_buurten.json"` -> the name and path of the GeoJson file
   - `data=df_grun` -> the geopandas dataframe containing addtional information of each neighbourhood<br>
     like the population density and number of inhabitants
   - `columns=\['BU_CODE','BEV_DICHTH'\]` -> we are using the neighbourhood code and population density<br>
     to display the density by color code
   - `tiles='OpenStreetMap'` -> use the open street map background
   - `key_on='feature.properties.BU_CODE'` -> this is the propertie in the GeoJson file to look for<br>
     when plotting the neighbourhood boundaries
   - `fill_color='RdPu'` -> use the color map from red to purple to display the population density
   - `fill_opacity=0.3` -> scale from 0 to 1, 0 = no fill, 1 = completely cover
3. Add circle shaped markers using the folium `CircleMarker` class to create a marker showing in<br>
   the central location of each neighbourhood. when a user hovers over the markerwith a mouse,<br>
   a text will appear showing the name, number of inhabitants and population density of the neighbourhood<br>
   **Method attributes**
   - `[lat, lon]` -> latitude and logitude position of the neighbourhood marker (centroid)
   - `radius=4` -> circle radius, a higher number is larger
   - `popup=label` -> display text in a popup when clicking on the marker. 
   - `tooltip=toolt` -> display text when the mouse moves over a marker
   - `color='red'` -> display the marker in read
   - `fill=True` -> fill the marker with a background color
   - `fill_opacity=0.3` -> set the markers opacity to 0.3

**Note** See the documentation for folium for further [information](https://python-visualization.github.io/folium/modules.html)

In [4]:
# 1. Use the central coordinates of the city of Groningen and use these to display the map
#    The zoom_start attribute is used for the zoom factor when initially displaying the map
map_groningen = folium.Map(location=[53.216116, 6.562249], zoom_start=13);

# 2. Set up the Choropleth map object
folium.Choropleth(geo_data="data/groningen_buurten.json",
    data = df_grun,
    popup=df_grun['BU_CODE'],
    columns=['BU_CODE','BEV_DICHTH'],
    key_on='feature.properties.BU_CODE',
    tiles='OpenStreetMap',
    fill_color='RdPu', 
    fill_opacity=0.3, 
    line_opacity=0.2,
    legend_name='Groningen - bevolkingsdichtheid per buurt - 2017').add_to(map_groningen)

# 3. Looping through each neighbourhood's attributes create circle-shaped markers which when a user hovers over the marker
#    with a mouse, a text will appear showing the name, number of inhabitants and population density of the neighbourhood
for lat, lon, name, density, population in zip(df_grun['Latitude'], df_grun['Longitude'], 
                                               df_grun['BU_NAAM'], df_grun['BEV_DICHTH'],df_grun['AANT_INW']):
    label = folium.Popup(html = '<h4>{}</h4><br> inhabitants: {} ({}/km2)'.format(name,str(population),str(density)), parse_html=False)
    toolt = '{} - inhabitants: {} ({}/km2)'.format(name, str(population),str(density))
    folium.CircleMarker(
        [lat, lon],
        radius=4,
        popup=label,
        tooltip=toolt,
        color='red',
        fill=True,
        # fill_color=rainbow[cluster-1],
        fill_opacity=0.3).add_to(map_groningen) 
map_groningen.save('data/bevolkings_dichtheid.html')

# display the map
map_groningen

## Simplest version displaying a map using folium's Map class
- create a map object based on folium's Map class<br>
  `map_groningen = folium.Map(location=[53.216116, 6.562249], zoom_start=12)`
- add the GeoJson file to the map<br>
  `folium.GeoJson("data/groningen_buurten.json").add_to(map_groningen)`
- display the map<br>
  `map_groningen`

In [5]:
map_groningen = folium.Map(location=[53.216116, 6.562249], zoom_start=12)
folium.GeoJson("data/groningen_buurten.json").add_to(map_groningen)
map_groningen